In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp
from scipy.special import expit
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from scipy.spatial import distance
import statsmodels.api as sm
import warnings
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats import rankdata
from sklearn.cross_decomposition import CCA
import math
import copy
from math import log
from collections import Counter
import pickle
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
warnings.filterwarnings("ignore")

In [3]:
with open('relation_matrix_dic.pkl', 'rb') as f:
    relation_matrix_dic = pickle.load(f)

# define a new adjacency_matrix_dic: adjacency_matrix_dic[i] = relation_matrix_dic[i].iloc[0:20, 0:20]
adjacency_matrix_dic = {i: relation_matrix_dic[i].iloc[0:5, 0:5] for i in relation_matrix_dic}

# Read beta back from the file
beta = np.load('beta.npy')
print("beta:", beta)

beta: [ 3.87593318  0.12303241 -0.82319167  2.8958156   4.70534406  0.4141024
  3.33198974  5.11773944  5.36558892  2.8544744 ]


In [5]:
len(adjacency_matrix_dic[0])

5

In [7]:
# follow the beta, only 1,2,3 related is non-zero
K = 9

In [9]:
# change beta into length 1 + 9 + 9 + choose 2 from 9
# with initial beta all 0
# beta[0:4] = beta[0:4] , i.e. 0,1,2,3
# beta[11:14] = beta[4:7], i.e. (1,1), (1,2), (1,3)
# beta[20 and 21] = beta[7 and 8]
# beta [28] = beta[9]

# Initialize a new beta array of length 55
new_beta = np.zeros(55)

# Assign values based on the provided instructions
new_beta[0:4] = beta[0:4]            # Copy the first four elements unchanged
new_beta[10:13] = beta[4:7]          # Copy beta[4:6] into new_beta[11:13]
new_beta[19:21] = beta[7:9]          # Copy beta[7:8] into new_beta[20:21]
new_beta[27] = beta[9]               # Copy beta[9] into new_beta[28]

# Print the new_beta array
print("new_beta:", new_beta)

beta = new_beta

new_beta: [ 3.87593318  0.12303241 -0.82319167  2.8958156   0.          0.
  0.          0.          0.          0.          4.70534406  0.4141024
  3.33198974  0.          0.          0.          0.          0.
  0.          5.11773944  5.36558892  0.          0.          0.
  0.          0.          0.          2.8544744   0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.        ]


In [11]:
for i in range(10):
    print(f"Position {i}: beta = {beta[i]}")

idx = 10

for i in range(1, 10):
    for j in range(1, 10):
        if i <= j:
            print(f"Position ({i},{j}): beta = {beta[idx]}")
            idx += 1

Position 0: beta = 3.8759331788989613
Position 1: beta = 0.12303240864427623
Position 2: beta = -0.8231916699248809
Position 3: beta = 2.895815598774753
Position 4: beta = 0.0
Position 5: beta = 0.0
Position 6: beta = 0.0
Position 7: beta = 0.0
Position 8: beta = 0.0
Position 9: beta = 0.0
Position (1,1): beta = 4.7053440610515
Position (1,2): beta = 0.4141024027012443
Position (1,3): beta = 3.331989738972724
Position (1,4): beta = 0.0
Position (1,5): beta = 0.0
Position (1,6): beta = 0.0
Position (1,7): beta = 0.0
Position (1,8): beta = 0.0
Position (1,9): beta = 0.0
Position (2,2): beta = 5.117739436696092
Position (2,3): beta = 5.365588921449674
Position (2,4): beta = 0.0
Position (2,5): beta = 0.0
Position (2,6): beta = 0.0
Position (2,7): beta = 0.0
Position (2,8): beta = 0.0
Position (2,9): beta = 0.0
Position (3,3): beta = 2.854474397693938
Position (3,4): beta = 0.0
Position (3,5): beta = 0.0
Position (3,6): beta = 0.0
Position (3,7): beta = 0.0
Position (3,8): beta = 0.0
Posit

In [13]:
# define a function named "counting_treatment"
def counting_treatment(adjacency_matrix, A, K):

    # adjacency_matrix is a adjacency matrix dataframe with length 20 * 20 known
    # A is a np.array with length 20, each element 1,2,...,K known corresponding to each node in the network

    # a empty df named counting_treatment with column named: 0,1,...,K, (1,1), ..., (1,K), (2,2), ..., (2,K),...(K,K)

    # add different numbers of different nodes and different edges. For the column 0, directly give 1.

    # Define column names
    basic_columns = list(range(0, K + 1))  # Including column 0
    edge_columns = [(i, j) for i in range(1, K + 1) for j in range(i, K + 1)]
    columns = basic_columns + edge_columns
    
    # Create an empty DataFrame
    counting_treatment_df = pd.DataFrame(columns=columns)
    counting_treatment_df.loc[0] = 0  # Initialize all counts to 0
    
    # Column 0 gets the value of 1 as per the description
    counting_treatment_df.at[0, 0] = 1

    # Count number of nodes with each treatment
    for treatment in range(1, K + 1):
        counting_treatment_df.at[0, treatment] = sum(A == treatment)
    
    # Count the number of edges connecting nodes of different treatments
    for i in range(len(adjacency_matrix_dic[0])):
        for j in range(i + 1, len(adjacency_matrix_dic[0])):  # Iterate only for j > i to avoid double-counting
            if adjacency_matrix.iloc[i, j] != 0:  # If there's an edge between node i and node j
                treatment_i = A[i]
                treatment_j = A[j]
                # Use sorted tuple to represent undirected edge
                edge = tuple(sorted((treatment_i, treatment_j)))
                if edge in counting_treatment_df.columns:
                    counting_treatment_df.at[0, edge] += 1
                
    return counting_treatment_df

# generate_outcome 

In [16]:
def generate_outcome(adjacency_matrix, A, K, beta, error):
    # Generate the treatment matrix using the counting_treatment function
    counting_treatment_df = counting_treatment(adjacency_matrix, A, K)

    # Convert the treatment matrix to a NumPy array for easier calculation
    counting_treatment_array = counting_treatment_df.values.flatten()

    # Ensure that beta and counting_treatment_array have the same length
    if len(beta) != len(counting_treatment_array):
        raise ValueError("Length of beta does not match the number of columns in counting_treatment.")

    # Compute the outcome Y
    Y_exp = np.dot(counting_treatment_array, beta)
    
    Y = np.dot(counting_treatment_array, beta) + error

    # Return the results
    return Y, Y_exp, counting_treatment_df

# lasso regression

In [18]:
def predict_lasso_model(experimented_data, lambda_value):
    # Step 1: Prepare Data (X, Y)
    # X consists of all data except 'Y' from experimented_data
    X = experimented_data.iloc[:, :-1].values  # Take all columns except the last one (assumed 'Y' is the last column)
    Y = experimented_data['Y'].values.reshape(-1, 1)  # Rewards (Y) as a column vector

    X_t = X
    Y_t = Y.ravel()

    ridge = Ridge(alpha=alpha_r, fit_intercept=False)
    ridge.fit(X_t, Y_t)  # Fit the model to get beta estimates
    beta_breve = ridge.coef_

    # Avoid division by zero for penalties
    beta_abs = np.abs(beta_breve)
    beta_abs[beta_abs == 0] = 1e-6  # Replace zeros with a small value to avoid division by zero
    
    # Custom penalty for each beta
    custom_penalty = lambda_value / beta_abs
    
    # Assume X_t and Y_t are input data, and custom_penalty is the weight for each parameter
    n_features = X_t.shape[1]  # Number of features in the input data
    beta = cp.Variable(n_features)  # Define the variable to be optimized (coefficients beta)
    
    # Define the objective function for Weighted Lasso
    lasso_loss = cp.norm(Y_t - X_t @ beta, 2) ** 2 / (2 * len(Y_t))  # Least squares loss term
    regularization = cp.sum(cp.multiply(custom_penalty, cp.abs(beta)))  # Weighted L1 regularization term
    objective = cp.Minimize(lasso_loss + regularization)  # Define the minimization objective
    
    # Define and solve the optimization problem
    problem = cp.Problem(objective)  # Create a problem instance with the defined objective
    problem.solve()  # Solve the optimization problem
    
    # Get the results
    beta_esti = beta.value  # The estimated coefficients (optimized beta values)
    s_esti = np.count_nonzero(beta_esti)  # Count the number of non-zero coefficients

    # Step 4: Compute empirical covariance matrix with regularized
    empirical_cov_matrix = (1 / X.shape[0]) * np.dot(X.T, X)
    
    return beta_esti, s_esti, empirical_cov_matrix

In [20]:
def predict_lasso_reward(A, beta_esti, empirical_cov_matrix, adjacency_matrix, K):

    # Predict for new target treatments
    # Generate new target treatments using counting_treatment function
    counting_treatment_df = counting_treatment(adjacency_matrix, A, K)  # Get DataFrame representation
    
    # Convert the dataframe to a NumPy array for GPy compatibility
    X_target = counting_treatment_df.values  # Convert the DataFrame to a NumPy array

    # Calculate the weighted norm before reshaping (for 1D vectors)
    if X_target.ndim == 1:
        # Direct calculation for 1D vector
        norm_2 = np.sqrt(np.dot(X_target, np.dot(np.linalg.pinv(empirical_cov_matrix), X_target)))
    else:
        # If already 2D, calculate as before
        norm_2 = np.sqrt(np.dot(X_target, np.dot(np.linalg.pinv(empirical_cov_matrix), X_target.T))[0, 0])
    
    # Ensure X_target has the correct shape (num_samples, num_features)
    if X_target.ndim == 1:
        X_target = X_target.reshape(1, -1)  # Reshape to be a 2D array with one row

    

    # Predict the rewards and the full covariance matrix for the new target treatments
    predicted_reward = np.dot(X_target, beta_esti)

    # Simplify the results to a scalar value if necessary
    # For Lasso, the output of .predict() should be a 1D array, so we can just return the first value
    
    return predicted_reward, norm_2


# get the parents for next generation by ucb

In [63]:
# optimization genetic algorithm
def get_children_elite(parent_treatment_dic, parent_reward_predict, beta_esti, empirical_cov_matrix, adjacency_matrix, K, generation_num, gama):
    child_treatment_dic = {}
    child_reward_predict = {}
    
    # Step 2: Generate children from parent treatments using roulette wheel selection
    for i in range(generation_num):
        # Use roulette wheel selection to choose two parents based on UCB values
        A1, A2 = roulette_wheel_selection(parent_reward_predict, parent_treatment_dic)
        
        # Crossover: with 0.5 probability choose genes from A1 or A2
        child_treatment_dic[i + generation_num] = np.where(np.random.rand(len(A1)) < 0.5, A1, A2)
        
        # Predict reward and UCB for the new child
        predicted_rewards, norm_2 = predict_lasso_reward(child_treatment_dic[i + generation_num], beta_esti, empirical_cov_matrix, adjacency_matrix, K)
        child_reward_predict[i + generation_num] = predicted_rewards + gama * norm_2

    # Step 3: Select the top treatments from both parents and children
    combined_predict = {**parent_reward_predict, **child_reward_predict}
    
    # Sort the combined dictionary by UCB values and select the top N treatments
    sorted_indices = sorted(combined_predict, key=combined_predict.get, reverse=True)[:generation_num]
    child_elite = {idx: combined_predict[idx] for idx in sorted_indices}
    
    child_elite_treatment_dic = {}
    for idx in sorted_indices:
        if idx in parent_reward_predict:
            child_elite_treatment_dic[idx] = parent_treatment_dic[idx]
        else:
            child_elite_treatment_dic[idx] = child_treatment_dic[idx]

    return child_elite_treatment_dic, child_elite


def roulette_wheel_selection(reward_predict, treatment_dic):

    values = np.array(list(reward_predict.values()))

    # Ensure no negative values by shifting if necessary
    if np.min(values) < 0:
        values = values - np.min(values)  # Shift values to make them non-negative
    
    total = np.sum(values)
    
    if total > 0:
        probabilities = values / total  # Normalize to create valid probabilities
    else:
        # If all UCB values are zero, use uniform probability
        probabilities = np.ones_like(values) / len(values)  # Equal probability if all UCB are zero

    # Select two parents based on the computed probabilities
    selected_indices = np.random.choice(len(values), size=2, p=probabilities.ravel(), replace=False)
    return treatment_dic[selected_indices[0]], treatment_dic[selected_indices[1]]

# get the next experiment by ucb

In [66]:
# get the next experiment

def get_next_experiment(beta_esti, empirical_cov_matrix, adjacency_matrix, K, generation_num, epoch, gama):
    parent_treatment_dic = {i: np.random.randint(1, K+1, size=adjacency_matrix.shape[0]) for i in range(100)}
    parent_reward_predict = {}
    
    # Step 1: Calculate UCB for parent treatments
    for i in range(generation_num):
        predicted_rewards, norm_2 = predict_lasso_reward(parent_treatment_dic[i], beta_esti, empirical_cov_matrix, adjacency_matrix, K)
        # UCB
        parent_reward_predict[i] = predicted_rewards + gama * norm_2

        
    
    # Epoch loop
    # Initialize variables
    no_progress_counter = 0  # Counter to track epochs without improvement
    best = -np.inf  # To store the best UCB value so far
    
    
    # Epoch loop
    for k in range(1, epoch + 1):
        # Get the elite children treatments and UCBs
        child_elite_treatment_dic, child_elite = get_children_elite(parent_treatment_dic, parent_reward_predict, beta_esti, empirical_cov_matrix, adjacency_matrix, K, generation_num, gama)
        
        # Reassign parent treatments and UCB predictions with renumbered indices
        parent_treatment_dic = {new_idx: treatment for new_idx, treatment in enumerate(child_elite_treatment_dic.values())}
        parent_reward_predict = {new_idx: sur for new_idx, sur in enumerate(child_elite.values())}
    
        # Get the best UCB in this epoch
        current_best = parent_reward_predict[0]
    
        # Print the UCB of the best treatment after each epoch
        # print(f"Epoch {k}: Best UCB = {current_best}")
    
        # Check if the best UCB has improved
        if current_best > best:
            best = current_best  # Update the best UCB
            no_progress_counter = 0  # Reset the counter
        else:
            no_progress_counter += 1  # No improvement, increment the counter
    
        # If no improvement for 10 epochs, break the loop
        if no_progress_counter >= 10:
            # print(f"No improvement for 10 consecutive epochs. Stopping at epoch {k}.")
            break

    return parent_treatment_dic[0]

# initial experiment 0: for a selected vector treatments, we get rewards

In [69]:
def get_reward_array(phi):

    generation_num = 100
    epoch = 100

    T = 100
    a = 2
    d = len(beta)
    sigma = 1
    lambda_value = 2 * sigma
    
    # Get the length of relation_matrix_dic (number of keys)
    length = len(relation_matrix_dic)
    
    # Generate the error array from a normal distribution with the specified length
    mean = 0  # Define the mean
    std_dev = sigma
    
    error_array = mean + std_dev * np.random.randn(length)


    # Initialize empty lists to store A and reward over time
    A_time = []
    reward_time = []
    
    # Create an array A , with length matching the adjacency matrix
    A = np.random.randint(1, K + 1, size=adjacency_matrix_dic[0].shape[0])
    
    Y, Y_exp, counting_treatment_df = generate_outcome(adjacency_matrix_dic[0], A, K, beta, error_array[0])
    
    # get experimented_data
    # Create a DataFrame for Y
    Y_df = pd.DataFrame([Y], columns=['Y'])
    # Concatenate treatment and Y_df to form experimented_data
    experimented_data = pd.concat([counting_treatment_df, Y_df], axis=1)
    
    A_time.append(A)
    reward_time.append(Y_exp)

    print(f"Time {0} with reward: {Y_exp}")

    for t in range(1,T):  # Iterate for T time steps

        beta_esti, non_zero_coefficients_count, empirical_cov_matrix = predict_lasso_model(experimented_data, lambda_value)
    
        # Calculate beta
        gama = 4 * np.sqrt(non_zero_coefficients_count) * lambda_value / ( phi )
        
        # Get the next treatment vector A
        A = get_next_experiment(beta_esti, empirical_cov_matrix, adjacency_matrix_dic[t], K, generation_num, epoch, gama)
        
        # Generate the outcome based on A
        Y, Y_exp, counting_treatment_df = generate_outcome(adjacency_matrix_dic[t], A, K, beta, error_array[t])
        
        # Create a DataFrame for Y
        Y_df = pd.DataFrame([Y], columns=['Y'])
        
        # Concatenate transformed_treatment and Y_df to form experimented_data
        experimented_data = pd.concat([experimented_data, pd.concat([counting_treatment_df, Y_df], axis=1)], ignore_index=True)
    
        # Append current treatment and reward to their respective lists
        A_time.append(A)
        reward_time.append(Y_exp)
    
        lambda_value = 2 * sigma * np.sqrt(2 * (a * np.log(t) + np.log(d)) / t)
    
        # Print the reward for the current time step
        print(f"Time {t} with reward: {Y}")
        # print(f"Time {t} with treatment: {A}")

    # Convert reward_time to a NumPy array
    reward_time = np.array(reward_time)

    return reward_time

In [76]:
# Values of phi to iterate over
phi_values = [30,60]

alpha_r = 2

# Loop through the different phi values
for phi in phi_values:

    Y_t_exp_times = []
    # Run the experiment 10 times
    for times in range(10):
        print(f"Running experiment {times + 1} for phi={phi}")

        Y_t_exp = get_reward_array(phi)

        # Append the results of this experiment
        Y_t_exp_times.append(Y_t_exp)
        
    
    # Convert lists to NumPy arrays for easier computation
    Y_t_exp_times = np.array(Y_t_exp_times)
    # Save the results to .npy files
    np.save(f'Y_t_exp_time-ada-lasso-stochastic-bandit-under-inter-n5-k9-d55-s10_10times.npy', Y_t_exp_times)

Running experiment 1 for phi=30
Time 0 with reward: 10.226770925290715
Time 1 with reward: 4.943170438261286
Time 2 with reward: 7.056285931230808
Time 3 with reward: 33.00877173583654
Time 4 with reward: 22.53446528458429
Time 5 with reward: 4.116747690365075
Time 6 with reward: 13.228265641871072
Time 7 with reward: 35.837060280901774
Time 8 with reward: 31.616220396590567
Time 9 with reward: 32.33684303862916
Time 10 with reward: 44.0707224080446
Time 11 with reward: 8.631941965481658
Time 12 with reward: 41.824614474994874
Time 13 with reward: 45.43210762616769
Time 14 with reward: 36.791703880451664
Time 15 with reward: 32.51322361135153
Time 16 with reward: 46.538532545017155
Time 17 with reward: 22.429701727879095
Time 18 with reward: 28.121145132884326
Time 19 with reward: 53.480606613968256
Time 20 with reward: 34.11974903260694
Time 21 with reward: 37.88911771029265
Time 22 with reward: 26.66769561637261
Time 23 with reward: 13.031722713996517
Time 24 with reward: 19.42067671